In [0]:
%run /Users/roboticsmanoj@gmail.com/retail_sales_project/Data_Validation

True

### Creating Temporary Views of all the ready to use validated data frames

In [0]:
sales_df_validated.createOrReplaceTempView("sales_table")
customers_df_validated.createOrReplaceTempView("customers_table")
products_df_validated.createOrReplaceTempView("products_table")
stores_df.createOrReplaceTempView("stores_table")
promotions_df_validated.createOrReplaceTempView("promotions_table")


+------------+-------------+----------+----------+-----------------+
|promotion_id|discount_rate|start_date|  end_date|Validation Errors|
+------------+-------------+----------+----------+-----------------+
|      400003|        28.71|2024-04-12|2024-11-26|                 |
|      400004|        80.49|2025-03-19|2025-07-12|                 |
|      400005|        48.96|2023-06-27|2023-10-27|                 |
|      400006|        79.15|2024-11-15|2025-07-02|                 |
|      400007|        82.49|2024-10-16|2024-11-20|                 |
|      400008|         1.89|2023-06-25|2024-01-07|                 |
|      400009|         42.2|2025-04-13|2026-02-28|                 |
|      400011|        27.74|2024-03-31|2025-01-05|                 |
|      400012|        82.77|2025-02-04|2025-07-14|                 |
|      400014|        88.56|2024-11-13|2025-10-01|                 |
+------------+-------------+----------+----------+-----------------+
only showing top 10 rows

+-------

### Business Scenario - 1 : Promotion Effectiveness Analysis - Comparing Total Sales and Average price of a product between promotion period and non-promotion period

In [0]:
discounts_worth = spark.sql("""
    WITH DISCOUNTED AS (
        SELECT SUM(s.amount - s.discount) AS TOTAL_PRICE,
               AVG(s.amount - s.discount) AS AVG_PRICE
        FROM sales_table s
        JOIN promotions_table p
            ON s.sale_date BETWEEN p.start_date AND p.end_date
    ),
    NOT_DISCOUNTED AS (
        SELECT SUM(s.amount) AS TOTAL_PRICE,
               AVG(s.amount) AS AVG_PRICE
        FROM sales_table s
        LEFT ANTI JOIN promotions_table p
            ON s.sale_date BETWEEN p.start_date AND p.end_date
    )
    SELECT 'WHEN DISCOUNTED' AS SCENARIO, TOTAL_PRICE, AVG_PRICE FROM DISCOUNTED
    UNION ALL
    SELECT 'WHEN NOT DISCOUNTED' AS SCENARIO, TOTAL_PRICE, AVG_PRICE FROM NOT_DISCOUNTED
""")

discounts_worth.show()


+-------------------+--------------+------------+
|           SCENARIO|   TOTAL_PRICE|   AVG_PRICE|
+-------------------+--------------+------------+
|    WHEN DISCOUNTED|36287481463.85|12025.757077|
|WHEN NOT DISCOUNTED| 2797804520.02|11987.833598|
+-------------------+--------------+------------+



### Business Scenario - 2 : Highest Discount of Every Product

In [0]:
  highest_discount = spark.sql("""
      WITH max_discounts AS(
          SELECT product_id , max(discount) AS max_discount
          FROM sales_table
          GROUP BY product_id    
      )
      SELECT s.product_id , s.max_discount , p.name
      FROM max_discounts s
      JOIN products_table p
      ON s.product_id == p.product_id
            
                              """)
  
  highest_discount.show()

+----------+------------+--------+
|product_id|max_discount|    name|
+----------+------------+--------+
|    200002|       29.91|  public|
|    200296|       29.71|audience|
|    200138|       29.76|  choose|
|    200206|       29.92|    open|
|    200853|       29.88|  expert|
|    200979|       29.97|anything|
|    200353|       29.80| someone|
|    200635|       30.00| control|
|    200962|       29.93|    base|
|    200878|       29.92|    best|
|    200663|       29.78| feeling|
|    200508|       29.78|   where|
|    200360|       29.85|     Mrs|
|    200720|       30.00| partner|
|    200495|       29.97|       I|
|    200096|       29.94|     out|
|    200037|       29.82|   stock|
|    200460|       29.83|    term|
|    200757|       29.96|    move|
|    200538|       29.85|  others|
+----------+------------+--------+
only showing top 20 rows

